# Assignment 1 - Rhythm Generator
### Mariella Daghfal, Kaede Johnson, Shayan Khajehnouri, & Tahel Singer

In [1]:
# to be safe created venv with anaconda terminal.
# conda create -n "digital_musicology" python=3.8
# conda activate digital_musicology
# conda install pip

In [2]:
## Dependencies
# pip install music21
from music21 import *

## for viewing score, download and install finale note pad at https://store.makemusic.com/Downloads/Default.aspx?id=538&_ga=2.3616692.58482559.1678537005-1850486829.1678537005 and then run:
# configure.run()

## I don't know if it's because I screwed up something during initial setup but this is what I had to do to get music showing:
# environment.set('musescoreDirectPNGPath', 'C:\\Program Files (x86)\\Finale NotePad 2012\\Finale NotePad.exe')

In [3]:
## Bach score from package

s = corpus.parse('bach/bwv65.2.xml')
s.show('text')

# image version
# s.show()

{0.0} <music21.text.TextBox 'PDF © 2004...'>
{0.0} <music21.text.TextBox 'BWV 65.2'>
{0.0} <music21.metadata.Metadata object at 0x180a5d22b50>
{0.0} <music21.stream.Part Soprano>
    {0.0} <music21.instrument.Instrument 'P1: Soprano: '>
    {0.0} <music21.stream.Measure 0 offset=0.0>
        {0.0} <music21.layout.SystemLayout>
        {0.0} <music21.clef.TrebleClef>
        {0.0} <music21.key.Key of C major>
        {0.0} <music21.meter.TimeSignature 3/4>
        {0.0} <music21.note.Note A>
    {1.0} <music21.stream.Measure 1 offset=1.0>
        {0.0} <music21.note.Note A>
        {2.0} <music21.note.Note A>
    {4.0} <music21.stream.Measure 2 offset=4.0>
        {0.0} <music21.note.Note B>
        {2.0} <music21.note.Note B>
    {7.0} <music21.stream.Measure 3 offset=7.0>
        {0.0} <music21.note.Note C>
        {1.0} <music21.note.Note B>
        {2.0} <music21.note.Note A>
    {10.0} <music21.stream.Measure 4 offset=10.0>
        {0.0} <music21.note.Note G>
    {13.0} <music21.st

In [7]:
## Load in first score from DH set

# Change folder below to xml file locations
xml_file_location = 'C:/Users/HP/anaconda3/envs/digital_musicology/'

# Pull in first sample
sample_score = converter.parse(xml_file_location + 'xml/Dahl_polska_efter_Ola_Olsson_bf94a6.xml')
print('music21 object for imported score is called ' + str(type(sample_score)))
print('sample score key is ' + str(sample_score.analyze('key')))


music21 object for imported score is called <class 'music21.stream.base.Score'>
sample score key is A major


In [78]:
print('The first time signature object is ' + sample_score.recurse().getElementsByClass(meter.TimeSignature)[0].ratioString)
print('There is (are) ' + str(len(sample_score.recurse().getElementsByClass(meter.TimeSignature))) + ' time signature object(s)')
print('Measure 6 does not have a time signature object: ' + str(sample_score.measure(6).timeSignature is None))

The first time signature object is 3/4
There is (are) 1 time signature object(s)
Measure 6 does not have a time signature object: True
3/4


In [79]:
print('Recall the sample score is in time ' + sample_score.recurse().getElementsByClass(meter.TimeSignature)[0].ratioString)
curr_measure = 0
for i in sample_score.recurse().notes:
    if i.measureNumber != curr_measure:
        curr_measure = i.measureNumber
        print("Measure " + str(curr_measure))
    print('Beat: ' + str(i.beat) + ', Duration: ' + str(i.duration.quarterLength) + ', Pitch: ' + str(i.name))

Recall the sample score is in time 3/4
Measure 1
Beat: 1.0, Duration: 0.5, Pitch: A
Beat: 1.5, Duration: 0.25, Pitch: G#
Beat: 1.75, Duration: 0.25, Pitch: A
Beat: 2.0, Duration: 1.5, Pitch: E
Beat: 3.5, Duration: 0.5, Pitch: D
Measure 2
Beat: 1.0, Duration: 0.5, Pitch: C#
Beat: 1.5, Duration: 0.25, Pitch: B
Beat: 1.75, Duration: 0.25, Pitch: C#
Beat: 2.0, Duration: 1.5, Pitch: A
Beat: 3.5, Duration: 0.5, Pitch: A
Measure 3
Beat: 1.0, Duration: 0.5, Pitch: A
Beat: 1.5, Duration: 0.5, Pitch: C#
Beat: 2.0, Duration: 1.0, Pitch: E
Beat: 3.0, Duration: 0.5, Pitch: G#
Beat: 3.5, Duration: 0.5, Pitch: F#
Measure 4
Beat: 1.0, Duration: 1.0, Pitch: F#
Beat: 2.0, Duration: 1.0, Pitch: E
Measure 5
Beat: 1.0, Duration: 0.5, Pitch: A
Beat: 1.5, Duration: 0.5, Pitch: A
Beat: 2.0, Duration: 0.5, Pitch: B
Beat: 2.5, Duration: 0.5, Pitch: B
Beat: 3.0, Duration: 0.5, Pitch: C#
Beat: 3.5, Duration: 0.5, Pitch: C#
Measure 6
Beat: 1.0, Duration: 0.5, Pitch: D
Beat: 1.5, Duration: 0.5, Pitch: D
Beat: 2.0, 

In [ ]:
# Tokenize...